In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
# https://www.kaggle.com/code/fuzzywizard/rec-sys-collaborative-filtering-dl-techniques

In [2]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering\Methylation_Model
df = pd.read_csv('NMI_Methylome_Male06_Age.csv')
df = df.drop(['PATNO', 'APPRDX'], axis = 1)
df.shape
df.head(2)

Z:\PPMI_Data\Excels\CollaborativeFiltering\Methylation_Model


,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,1,65.142466,0.023520,0.344298,0.021304,0.960123,0.821042,0.824603,0.038859,0.776180,...,0.835011,0.916727,0.925943,0.017127,0.895680,0.026921,0.873614,0.873463,0.458258,0.845385
1,2,64.547945,0.033311,0.306851,0.012358,0.934346,0.823265,0.875819,0.019836,0.366488,...,0.811623,0.912079,0.915694,0.018519,0.928806,0.033420,0.832723,0.863693,0.445499,0.834479


In [3]:
df2 = pd.read_csv('Methylation_Male_Empty.csv')
df3 = df2.drop('Patient_ID', axis = 1)
df3.head(2)

,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,cg00605793,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Ask the user for the updated value of age
new_age = int(input("What is the user's age (in years): "))

# Update the value of age in the dataframe
df3.at[0, 'Age'] = new_age

# Drop the 46th column from the original DataFrame
col_age = df3.pop('Age')

# Insert the column at the 5th position
df3.insert(0, "Age", col_age)
df3.head(2)

What is the user's age (in years): 44


,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Below will take at least 3 features

In [6]:
def update_values(df3):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 327): "))
    
    while num_columns < 3 or num_columns > 327:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 327): "))
    
    columns = []
    values = []
     
     # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the score of the Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(float(value))
    
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i]
    
    return df3

# Call the function
df3 = update_values(df3)

# Print the updated dataframe
df3.head()

Enter the number of columns you want to update (minimum 3, maximum 327): 3
Enter the name of the Feature 1: cg00128386
Enter the score of the Feature 1: 0.2
Enter the name of the Feature 2: cg00732415
Enter the score of the Feature 2: 0.1
Enter the name of the Feature 3: cg00017221
Enter the score of the Feature 3: 0.5


,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,44.0,0.5,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Adding back the column (as the first column)
df3.insert (0, 'Patient_ID', df2['Patient_ID'])
df3.head()

,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,999,44.0,0.5,NaN,NaN,0.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# merging two dataframes [original and our input dataframes]
merged_df = pd.concat ([df, df3], axis = 0)
merged_df.reset_index(inplace = True)
merged_df.tail()

,index,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
304,304,305,73.550685,0.022174,0.138859,0.014338,0.959909,0.910762,0.898318,0.030028,...,0.842528,0.921718,0.939466,0.045955,0.920351,0.020263,0.882894,0.917974,0.459722,0.913242
305,305,306,72.038356,0.031695,0.458419,0.015531,0.940823,0.865159,0.878984,0.030574,...,0.850479,0.909478,0.938597,0.014201,0.898545,0.019369,0.863895,0.853660,0.547316,0.872831
306,306,307,67.221918,0.015719,0.380722,0.010514,0.963340,0.899114,0.890893,0.038055,...,0.852996,0.908892,0.933013,0.018602,0.903400,0.011374,0.862685,0.764067,0.440161,0.906936
307,307,308,71.117808,0.026659,0.195581,0.014547,0.958277,0.865897,0.844592,0.022062,...,0.838157,0.929789,0.931938,0.017685,0.886063,0.019583,0.856798,0.897929,0.435785,0.863049
308,0,999,44.000000,0.500000,NaN,NaN,0.200000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# dropping a few columns
merged_df1 = merged_df.drop(['index','Patient_ID'], axis = 1)
merged_df1.head(2)

,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,65.142466,0.023520,0.344298,0.021304,0.960123,0.821042,0.824603,0.038859,0.776180,0.653704,...,0.835011,0.916727,0.925943,0.017127,0.895680,0.026921,0.873614,0.873463,0.458258,0.845385
1,64.547945,0.033311,0.306851,0.012358,0.934346,0.823265,0.875819,0.019836,0.366488,0.732829,...,0.811623,0.912079,0.915694,0.018519,0.928806,0.033420,0.832723,0.863693,0.445499,0.834479


In [10]:
# scaling values of every column from 1 to 5 [column wise]
scaler = MinMaxScaler (feature_range = (1,5))
df_scaled = scaler.fit_transform(merged_df1)
df_scaled = pd.DataFrame (df_scaled, columns = merged_df1.columns)
df_scaled.head(2)

,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,3.623305,1.091507,3.021661,1.557717,4.923385,4.109690,2.920225,3.160969,4.709588,3.494182,...,4.178653,3.705978,3.704015,1.563004,3.609501,1.988724,4.252401,4.570826,2.908658,3.615216
1,3.577409,1.171826,2.737435,1.169041,4.790340,4.130637,4.117010,2.089871,2.177625,4.007752,...,3.890290,3.481184,3.306073,1.663090,4.418021,2.333686,3.396316,4.491108,2.786893,3.424406


In [11]:
# adding the column [as the first column]
df_scaled.insert (0, 'Patient_ID', merged_df['Patient_ID'], True)
df_scaled.head(3)

,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,1,3.623305,1.091507,3.021661,1.557717,4.923385,4.109690,2.920225,3.160969,4.709588,...,4.178653,3.705978,3.704015,1.563004,3.609501,1.988724,4.252401,4.570826,2.908658,3.615216
1,2,3.577409,1.171826,2.737435,1.169041,4.790340,4.130637,4.117010,2.089871,2.177625,...,3.890290,3.481184,3.306073,1.663090,4.418021,2.333686,3.396316,4.491108,2.786893,3.424406
2,3,2.220098,1.137768,4.158035,1.199848,4.843357,3.844409,4.072871,2.127932,2.082193,...,3.683444,3.413333,3.071651,1.734282,4.131837,1.949099,3.074774,4.382538,3.035820,2.772327


In [12]:
df_scaled.tail(3)

,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
306,307,3.783834,1.027522,3.298113,1.088907,4.939994,4.845464,4.469243,3.115701,4.689441,...,4.400402,3.327066,3.978534,1.669120,3.797917,1.163421,4.023600,3.678200,2.735955,4.692158
307,308,4.084588,1.117260,1.892906,1.264135,4.913857,4.532419,3.387327,2.215226,3.649033,...,4.217435,4.337646,3.936806,1.603123,3.374767,1.599194,3.900334,4.770459,2.694193,3.924275
308,999,1.991151,5.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# using melt function [reshape]
df_melted = df_scaled.melt(id_vars='Patient_ID', var_name='Feature', value_name='Value')
print(df_melted.shape)
df_melted.head(2)

(101352, 3)


,Patient_ID,Feature,Value
0,1,Age,3.623305
1,2,Age,3.577409


In [14]:
#1. take groupby with respect to Feature and get mean avg of each Feature. 
feat_rating_count = df_melted.groupby('Patient_ID')['Value'].count()

#2. convert into dataframe for better operation work.
feat_rating_count = pd.DataFrame(feat_rating_count)

#3. change column name.
feat_rating_count.columns = ['Value_count']

#4. create new column Patient_ID
feat_rating_count['Patient_ID'] = feat_rating_count.index
feat_rating_count = feat_rating_count.reset_index(drop=True)

In [15]:
#merging to Rating dataframe so we have collectively all information together..
dframe = df_melted.merge(feat_rating_count, on = 'Patient_ID', how = 'inner')
print(dframe.shape)
dframe.head(2)

(101352, 4)


,Patient_ID,Feature,Value,Value_count
0,1,Age,3.623305,328
1,1,cg00017221,1.091507,328


In [16]:
#create a pivot matrix for user and Feature based on rating value..
RatingMat = dframe.pivot_table(index=['Patient_ID'],columns=['Feature'],values=['Value'],fill_value=0)

#keep a copy of original matrix for future refrence..
Original_RatingMat = RatingMat.copy()
RatingMat.head()

Value                                                         \
Feature          Age cg00017221 cg00030584 cg00113363 cg00128386 cg00156743   
Patient_ID                                                                    
1           3.623305   1.091507   3.021661   1.557717   4.923385   4.109690   
2           3.577409   1.171826   2.737435   1.169041   4.790340   4.130637   
3           2.220098   1.137768   4.158035   1.199848   4.843357   3.844409   
4           3.135160   1.044137   2.384265   1.151412   4.852242   4.738521   
5           2.666161   1.039057   2.477220   1.146941   4.972178   4.579327   

                                                        ...             \
Feature    cg00295303 cg00418511 cg00558297 cg00597128  ... cg26879084   
Patient_ID                                              ...              
1            2.920225   3.160969   4.709588   3.494182  ...   4.178653   
2            4.117010   2.089871   2.177625   4.007752  ...   3.890290   
3            4.072871   2.127932   2.082193   3.824850  ...   3.683444   
4            3.626483   2.547647   4.252528   4.901969  ...   4.550551   
5            2.921274   2.200981   4.409296   4.408771  ...   4.935363   

                                                                              \
Feature    cg26926318 cg26941073 cg27079446 cg27175112 cg27281389 cg27339255   
Patient_ID                                                                     
1            3.705978   3.704015   1.563004   3.609501   1.988724   4.252401   
2            3.481184   3.306073   1.663090   4.418021   2.333686   3.396316   
3            3.413333   3.071651   1.734282   4.131837   1.949099   3.074774   
4            4.451655   5.000000   4.575040   4.224583   1.619878   4.829476   
5            3.617834   4.434145   1.267615   3.746225   1.322808   4.488822   

                                             
Feature    cg27366263 cg27603338 cg27648238  
Patient_ID                                   
1            4.570826   2.908658   3.615216  
2            4.491108   2.786893   3.424406  
3            4.382538   3.035820   2.772327  
4            4.754072   3.426528   4.217822  
5            4.724553   3.113229   3.694648  

[5 rows x 328 columns]

In [17]:
#drop multilevel column for better usabiltity...
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

Feature,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,3.623305,1.091507,3.021661,1.557717,4.923385,4.109690,2.920225,3.160969,4.709588,3.494182,...,4.178653,3.705978,3.704015,1.563004,3.609501,1.988724,4.252401,4.570826,2.908658,3.615216
2,3.577409,1.171826,2.737435,1.169041,4.790340,4.130637,4.117010,2.089871,2.177625,4.007752,...,3.890290,3.481184,3.306073,1.663090,4.418021,2.333686,3.396316,4.491108,2.786893,3.424406
3,2.220098,1.137768,4.158035,1.199848,4.843357,3.844409,4.072871,2.127932,2.082193,3.824850,...,3.683444,3.413333,3.071651,1.734282,4.131837,1.949099,3.074774,4.382538,3.035820,2.772327
4,3.135160,1.044137,2.384265,1.151412,4.852242,4.738521,3.626483,2.547647,4.252528,4.901969,...,4.550551,4.451655,5.000000,4.575040,4.224583,1.619878,4.829476,4.754072,3.426528,4.217822
5,2.666161,1.039057,2.477220,1.146941,4.972178,4.579327,2.921274,2.200981,4.409296,4.408771,...,4.935363,3.617834,4.434145,1.267615,3.746225,1.322808,4.488822,4.724553,3.113229,3.694648


In [18]:
# Subtracting the mean rating of each user from their ratings
RatingMat_centered = RatingMat.sub(RatingMat.mean(axis=1), axis=0)

# User-wise similarity with centered cosine
user_similarity = cosine_similarity(RatingMat_centered)
user_sim_df = pd.DataFrame(user_similarity, index=RatingMat.index, columns=RatingMat.index)
user_sim_df.head(5)

Patient_ID,1,2,3,4,5,6,7,8,9,10,...,300,301,302,303,304,305,306,307,308,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.468867,0.457420,0.643814,0.737062,0.695394,0.663781,0.687247,0.695334,0.675514,...,0.673826,0.689374,0.619465,0.645321,0.724205,0.711514,0.712883,0.608019,0.717725,-0.060154
2,0.468867,1.000000,0.854971,0.674899,0.613272,0.738089,0.668709,0.680406,0.635075,0.600925,...,0.623895,0.734353,0.719614,0.694910,0.682235,0.657387,0.610889,0.527590,0.711930,-0.058196
3,0.457420,0.854971,1.000000,0.620796,0.592632,0.704305,0.651487,0.641783,0.590495,0.549695,...,0.565021,0.708148,0.689170,0.655101,0.627265,0.605393,0.574921,0.511677,0.657884,-0.085853
4,0.643814,0.674899,0.620796,1.000000,0.835544,0.829534,0.871171,0.845473,0.826209,0.710961,...,0.864993,0.819678,0.750420,0.820947,0.877443,0.888177,0.782540,0.686211,0.861758,-0.081445
5,0.737062,0.613272,0.592632,0.835544,1.000000,0.838578,0.854534,0.900407,0.773786,0.789515,...,0.842111,0.836925,0.814055,0.785222,0.854639,0.830783,0.861810,0.786473,0.871625,-0.076979


In [19]:
# user wise similarity with Euclidean distance
user_Eucsimilarity = 1 - pairwise_distances(RatingMat, metric="euclidean")
user_Eucsim_df = pd.DataFrame(user_Eucsimilarity, index=RatingMat.index, columns=RatingMat.index)
user_Eucsim_df.head(5)

Patient_ID,1,2,3,4,5,6,7,8,9,10,...,300,301,302,303,304,305,306,307,308,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-17.672230,-17.901710,-17.282497,-13.971770,-14.376056,-15.863262,-14.887296,-13.754573,-14.377563,...,-15.156793,-13.614379,-15.514214,-14.832246,-13.959770,-14.486965,-13.270697,-16.663398,-13.584417,-59.089135
2,-17.672230,1.000000,-8.447295,-16.559214,-16.762818,-13.125048,-15.571906,-14.834695,-14.678028,-15.523987,...,-16.058771,-12.187730,-12.758982,-13.245396,-14.834876,-15.613175,-15.176845,-17.867226,-13.495668,-58.134738
3,-17.901710,-8.447295,0.999999,-17.906126,-17.267236,-14.053349,-16.049522,-15.775059,-15.595785,-16.507980,...,-17.381412,-12.894120,-13.405195,-14.118373,-16.165826,-16.861199,-15.939891,-18.143547,-14.825705,-57.634714
4,-17.282497,-16.559214,-17.906126,1.000000,-12.228540,-12.008907,-10.710863,-11.797446,-12.994606,-16.460162,...,-10.305285,-12.366342,-15.703313,-13.312256,-10.022101,-9.282434,-13.780885,-17.485828,-10.897078,-65.184966
5,-13.971770,-16.762818,-17.267236,-12.228540,0.999998,-11.034789,-10.726720,-8.575891,-13.125344,-12.833139,...,-11.107220,-10.898898,-12.197370,-12.818864,-10.570053,-11.665207,-10.073095,-13.258623,-9.697513,-61.261712


In [20]:
# 1. select the user we have selected Patient_ID 999 
#Get a list of Feature user already watched so we can remove from our recommendation list..

current_user_rating = dframe[(dframe.Patient_ID == 999) & (dframe.Value != 0)]['Feature']
current_user_rating = pd.DataFrame(current_user_rating, columns=['Feature'])

In [21]:
print(user_sim_df.shape)
print(user_Eucsim_df.shape)

(309, 309)
(309, 309)


In [22]:
# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..

curr_user_similarity = pd.DataFrame(user_sim_df.loc[999])

#just changing column name as similarity for better readability.
curr_user_similarity.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_similarity.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_similarity1 = curr_user_similarity.iloc[1:, :]

# Reset index to move it to the first column
curr_user_similarity1.reset_index(inplace=True)

# Rename the index column
curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)

# Define a function to categorize values based on the condition
def categorize(x):
    if x < 215:
        return 'Patient'
    else:
        return 'Healthy'

# Apply the function to the first column of the dataframe
curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)

# Show the resulting dataframe
curr_user_similarity1.head()


C:\Users\system 4\AppData\Local\Temp\ipykernel_4108\4070062729.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)
C:\Users\system 4\AppData\Local\Temp\ipykernel_4108\4070062729.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)


,Patient_ID,Similarity_Score,Patient_Type
0,36,0.023844,Patient
1,10,-0.003701,Patient
2,286,-0.010811,Healthy
3,242,-0.012240,Healthy
4,64,-0.012395,Patient


The Patient_ID most similar to the input user is showing, from 5 we can decide patient or healthy based on 
more number of healthy or patient among those top 5 list generated above.

In [23]:
similar_user = curr_user_similarity1.iloc[:5, :]
counts = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts.index[0]
print("The user might fall under category of", most_occurring_value)

The user might fall under category of Patient


#As per the user category healthy or PD, we will give values to other unfilled features by that user

# Predicting Scoring of user: Top users of its category

In [24]:
select_simil_user = similar_user.loc[similar_user['Patient_Type'] == most_occurring_value]

In [25]:
df_simil = df[df['Patient_ID'].isin(select_simil_user['Patient_ID'])]
df_simil.head()

,Patient_ID,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
9,10,74.223827,0.110591,0.473314,0.015983,0.941025,0.885780,0.900107,0.022511,0.525973,...,0.807688,0.914464,0.920120,0.012503,0.898273,0.020170,0.864951,0.796687,0.428200,0.895032
35,36,69.865753,0.236515,0.172278,0.100528,0.914781,0.491076,0.848185,0.027292,0.604094,...,0.802114,0.916691,0.941794,0.037919,0.891222,0.015442,0.885875,0.889293,0.480947,0.860717
63,64,78.016146,0.023896,0.098277,0.013494,0.826364,0.887935,0.851290,0.028388,0.535262,...,0.577203,0.902740,0.874685,0.015632,0.924706,0.013787,0.738265,0.768328,0.359328,0.878203


In [26]:
similar_user.head()

,Patient_ID,Similarity_Score,Patient_Type
0,36,0.023844,Patient
1,10,-0.003701,Patient
2,286,-0.010811,Healthy
3,242,-0.012240,Healthy
4,64,-0.012395,Patient


In [27]:
similar_user = similar_user.drop(['Patient_Type'], axis=1)
similar_user.head(2)

,Patient_ID,Similarity_Score
0,36,0.023844
1,10,-0.003701


In [28]:
df_score = pd.merge(similar_user, df_simil, on = 'Patient_ID', how = 'inner')
df_score.head()

,Patient_ID,Similarity_Score,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,36,0.023844,69.865753,0.236515,0.172278,0.100528,0.914781,0.491076,0.848185,0.027292,...,0.802114,0.916691,0.941794,0.037919,0.891222,0.015442,0.885875,0.889293,0.480947,0.860717
1,10,-0.003701,74.223827,0.110591,0.473314,0.015983,0.941025,0.885780,0.900107,0.022511,...,0.807688,0.914464,0.920120,0.012503,0.898273,0.020170,0.864951,0.796687,0.428200,0.895032
2,64,-0.012395,78.016146,0.023896,0.098277,0.013494,0.826364,0.887935,0.851290,0.028388,...,0.577203,0.902740,0.874685,0.015632,0.924706,0.013787,0.738265,0.768328,0.359328,0.878203


In [ ]:
# Example DataFrame Y with some filled and some NaN values
X = df_simil
Y = df3.drop(['Patient_ID', 'Age'], axis = 1)

# Extract the weights from the second column of df_score i.e. extracting Similarity_Score
weights = df_score.iloc[:, 1]

# Extract the values to be averaged from the remaining columns of X
values = X.iloc[:, 2:]

# Compute the weighted average of each column
weighted_avg_cols = np.average(values, axis=0, weights=weights)

# Create a dictionary that maps column names to weighted average values
fill_values = {col: val for col, val in zip(Y.columns, weighted_avg_cols)}

# Fill NaN values in DataFrame Y with the mean values
Y = Y.fillna(fill_values)
col_Age = df3.pop('Age')
Y.insert(0, 'Age', col_Age)
Y

In [29]:
colX = df_simil
colY = df3.drop(['Patient_ID', 'Age'], axis = 1)
col_weights = df_score.iloc[:, 1]
values = colX.iloc[:, 2:]
weighted_avg_cols = np.average(values, axis=0, weights=col_weights)
fill_values = {col: val for col, val in zip(colY.columns, weighted_avg_cols)}
colY = colY.fillna(fill_values)
col_Age = df3.pop('Age')
colY.insert(0, 'Age', col_Age)
colY

,Age,cg00017221,cg00030584,cg00113363,cg00128386,cg00156743,cg00295303,cg00418511,cg00558297,cg00597128,...,cg26879084,cg26926318,cg26941073,cg27079446,cg27175112,cg27281389,cg27339255,cg27366263,cg27603338,cg27648238
0,44.0,0.5,1.217144,-0.020424,0.2,1.070887,1.003014,0.010825,0.473792,0.440008,...,1.179105,0.932156,0.982458,-0.004646,0.85935,0.032641,1.057639,0.797819,0.513191,0.938351
